In [1]:
import usb.core
import usb.util
import struct
import json
import canpack
import ctypes
import canalystii
import serial
import time
import pandas as pd
import plotly.express as px
import os
import json
import re

from ctypes import (
    c_bool,
    c_byte,
    c_ubyte,
    c_uint16,
    c_uint32,
    LittleEndianStructure,
    sizeof,
)


In [3]:

dev = canalystii.CanalystDevice(bitrate=500000)

1240 83


In [4]:
        
def dataInputBuffer(i):
    data = []
    try:
        data.append(dev.receive(i))
    except canalystii.NoData:
        pass
    
    return data

def send_message(raw_bytes):
    c_ubyte_array = (ctypes.c_ubyte * 8)(*raw_bytes)
        
    packed = canalystii.Message(can_id=0x300,
                                remote=False,
                                extended=False,
                                data_len=8,
                                data=(c_ubyte_array))
    dev.send(1, packed)

def test(message : str):
    d = None
    mc = canpack.MessageConstructor()
    m_array = mc.create_message(message)
    reconstructed_message = ''
    for m in m_array:
        letters_received = canpack.Message.fromBytesToMessage(m)
        reconstructed_message += letters_received.chuck.decode('ascii')
        print('Sent: ', letters_received)
        # print(hex(int.from_bytes(m)))
        send_message(m)
    for i in range(0, len(m_array*2)):
        print('i: ', i)
        try:
            received_message = dataInputBuffer(0)
            str = received_message.__str__()
            matches = re.findall(r'Data=([a-fA-F0-9]+)', str, re.IGNORECASE)
            
            for match in matches:
                print('match: ', match)
                byte_array = bytes.fromhex(match)
                #convert back to messgae
                d = canpack.Message.fromBytesToMessage(byte_array)
                print('Received: ', d)
                #check if checksum is correct
                if d.checksum == mc.checksum(d.chuck.decode('ascii')):
                    print('Checksum is correct')
                else:
                    exit('Checksum is incorrect')
                    #this will kill everything
                
                
        except IndexError:
            print('No data received')
                
        
        print(received_message)
    print('Original message: ', message)
    print('Reconstructed message: ', reconstructed_message)
    return d

In [84]:
d = dataInputBuffer(0)[0]


In [70]:
print((d[0]._fields_[7]))

('data', <class 'canalystii.protocol.c_ubyte_Array_8'>)


In [26]:
for i in range(100):
    print(r'[CanalystMessage ID=0x300 TS=0x2e87338 Data=726c642163010300]')

[CanalystMessage ID=0x300 TS=0x2e87338 Data=726c642163010300]
[CanalystMessage ID=0x300 TS=0x2e87338 Data=726c642163010300]
[CanalystMessage ID=0x300 TS=0x2e87338 Data=726c642163010300]
[CanalystMessage ID=0x300 TS=0x2e87338 Data=726c642163010300]
[CanalystMessage ID=0x300 TS=0x2e87338 Data=726c642163010300]
[CanalystMessage ID=0x300 TS=0x2e87338 Data=726c642163010300]
[CanalystMessage ID=0x300 TS=0x2e87338 Data=726c642163010300]
[CanalystMessage ID=0x300 TS=0x2e87338 Data=726c642163010300]
[CanalystMessage ID=0x300 TS=0x2e87338 Data=726c642163010300]
[CanalystMessage ID=0x300 TS=0x2e87338 Data=726c642163010300]
[CanalystMessage ID=0x300 TS=0x2e87338 Data=726c642163010300]
[CanalystMessage ID=0x300 TS=0x2e87338 Data=726c642163010300]
[CanalystMessage ID=0x300 TS=0x2e87338 Data=726c642163010300]
[CanalystMessage ID=0x300 TS=0x2e87338 Data=726c642163010300]
[CanalystMessage ID=0x300 TS=0x2e87338 Data=726c642163010300]
[CanalystMessage ID=0x300 TS=0x2e87338 Data=726c642163010300]
[Canalys

In [85]:
print(d)

[CanalystMessage ID=0x300 TS=0x2e87338 Data=726c642163010300]


In [87]:
str = d.__str__()
#[CanalystMessage ID=0x300 TS=0x2b5b800 Data=726c642163010300]
str_hex = str.split('Data=')[1].split(']')[0]
#convert hex string to bytes
bytes_object = bytes.fromhex(str_hex)
print(hex(int.from_bytes(bytes_object, byteorder='big')))

0x726c642163010300


In [88]:
print(bytes_object)

b'rld!c\x01\x03\x00'


In [254]:
dev = canalystii.CanalystDevice(bitrate=500000)

Exception ignored in: <function CanalystDevice.__del__ at 0x00000149D5144220>
Traceback (most recent call last):
  File "c:\Python311\Lib\site-packages\canalystii\device.py", line 103, in __del__
    self._dev.reset()
  File "c:\Python311\Lib\site-packages\usb\core.py", line 961, in reset
    self._ctx.backend.reset_device(self._ctx.handle)
  File "c:\Python311\Lib\site-packages\usb\backend\libusb0.py", line 618, in reset_device
    _check(_lib.usb_reset(dev_handle))
  File "c:\Python311\Lib\site-packages\usb\backend\libusb0.py", line 447, in _check
    raise USBError(errmsg, ret)
usb.core.USBError: [Errno None] b'libusb0-dll:err [reset] could not reset device, win error: The device does not recognize the command.\r\n\n'


1240 83


In [22]:
for i in range(1):
    test('Hello World!')

Sent:  Chuck: b'Hell', Checksum: 389, Counter: 1
Sent:  Chuck: b'o Wo', Checksum: 341, Counter: 2
Sent:  Chuck: b'rld!', Checksum: 355, Counter: 3
i:  0
[[]]
i:  1
[[]]
i:  2
[[]]
i:  3
[[]]
i:  4
[[]]
i:  5
[[]]
Original message:  Hello World!
Reconstructed message:  Hello World!


In [31]:
ser = serial.Serial('COM11', 9600, timeout=1)

In [32]:
ser.flushInput()
ser.flushOutput()
#d

In [33]:
df = pd.DataFrame(columns=["dht_temp", "dht_hum", "ccs811_co2", "ccs811_tvoc", "ccs811_temp"])
linesToRead = 10
for i in range(0, linesToRead):
    try:
        line = ser.readline().decode('utf-8')
        parts = line.split(',')
        if len(parts) == 5:
            df = df.append({'dht_temp': float(parts[0]), 'dht_hum': float(parts[1]), 'ccs811_co2': float(parts[2]), 'ccs811_tvoc': float(parts[3]), 'ccs811_temp': float(parts[4])}, ignore_index=True)
        print(line)
    except Exception as e:
        print(e)
        continue
display(df)

,dht_temp,dht_hum,ccs811_co2,ccs811_tvoc,ccs811_temp


In [34]:
linesToRead = 10
for i in range(0, linesToRead):
    try:
        line = ser.readline().decode('utf-8')
        parts = line.split(',')
        if len(parts) == 5:
            df = df.append({'dht_temp': float(parts[0]), 'dht_hum': float(parts[1]), 'ccs811_co2': float(parts[2]), 'ccs811_tvoc': float(parts[3]), 'ccs811_temp': float(parts[4])}, ignore_index=True)
        print(line)
    except Exception as e:
        print(e)
        continue


DHTxx test!

49.00,29.00,0,0,25.00

49.00,29.00,0,0,21.33

49.00,29.00,0,0,32.62

48.00,28.70,400,0,37.68



C:\Users\celes\AppData\Local\Temp\ipykernel_24852\1240850715.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'dht_temp': float(parts[0]), 'dht_hum': float(parts[1]), 'ccs811_co2': float(parts[2]), 'ccs811_tvoc': float(parts[3]), 'ccs811_temp': float(parts[4])}, ignore_index=True)
C:\Users\celes\AppData\Local\Temp\ipykernel_24852\1240850715.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'dht_temp': float(parts[0]), 'dht_hum': float(parts[1]), 'ccs811_co2': float(parts[2]), 'ccs811_tvoc': float(parts[3]), 'ccs811_temp': float(parts[4])}, ignore_index=True)
C:\Users\celes\AppData\Local\Temp\ipykernel_24852\1240850715.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'d

48.00,28.70,400,0,43.55



C:\Users\celes\AppData\Local\Temp\ipykernel_24852\1240850715.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'dht_temp': float(parts[0]), 'dht_hum': float(parts[1]), 'ccs811_co2': float(parts[2]), 'ccs811_tvoc': float(parts[3]), 'ccs811_temp': float(parts[4])}, ignore_index=True)


48.00,28.70,406,0,30.17


48.00,28.80,400,0,25.56



C:\Users\celes\AppData\Local\Temp\ipykernel_24852\1240850715.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'dht_temp': float(parts[0]), 'dht_hum': float(parts[1]), 'ccs811_co2': float(parts[2]), 'ccs811_tvoc': float(parts[3]), 'ccs811_temp': float(parts[4])}, ignore_index=True)


48.00,28.80,400,0,27.86



C:\Users\celes\AppData\Local\Temp\ipykernel_24852\1240850715.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'dht_temp': float(parts[0]), 'dht_hum': float(parts[1]), 'ccs811_co2': float(parts[2]), 'ccs811_tvoc': float(parts[3]), 'ccs811_temp': float(parts[4])}, ignore_index=True)


In [35]:
display(df)

,dht_temp,dht_hum,ccs811_co2,ccs811_tvoc,ccs811_temp
0,49.0,29.0,0.0,0.0,25.00
1,49.0,29.0,0.0,0.0,21.33
2,49.0,29.0,0.0,0.0,32.62
3,48.0,28.7,400.0,0.0,37.68
4,48.0,28.7,400.0,0.0,43.55
5,48.0,28.7,406.0,0.0,30.17
6,48.0,28.8,400.0,0.0,25.56
7,48.0,28.8,400.0,0.0,27.86


In [ ]:
while True:
    try:
        print(ser.readline().decode('utf-8'))

In [ ]:
import os, sys, re

with("t.txt",

In [108]:
#iterare per row
for index, row in df.iterrows():
    json_data = row.to_json()
    print(json_data)
    test(json_data)


{"dht_temp":49.0,"dht_hum":29.0,"ccs811_co2":0.0,"ccs811_tvoc":0.0,"ccs811_temp":25.0}
Sent:  Chuck: b'{"dh', Checksum: 361, Counter: 1
Sent:  Chuck: b't_te', Checksum: 428, Counter: 2
Sent:  Chuck: b'mp":', Checksum: 313, Counter: 3
Sent:  Chuck: b'49.0', Checksum: 203, Counter: 4
Sent:  Chuck: b',"dh', Checksum: 282, Counter: 5
Sent:  Chuck: b't_hu', Checksum: 432, Counter: 6
Sent:  Chuck: b'm":2', Checksum: 251, Counter: 7
Sent:  Chuck: b'9.0,', Checksum: 195, Counter: 8
Sent:  Chuck: b'"ccs', Checksum: 347, Counter: 9
Sent:  Chuck: b'811_', Checksum: 249, Counter: 10
Sent:  Chuck: b'co2"', Checksum: 294, Counter: 11
Sent:  Chuck: b':0.0', Checksum: 200, Counter: 12
Sent:  Chuck: b',"cc', Checksum: 276, Counter: 13
Sent:  Chuck: b's811', Checksum: 269, Counter: 14
Sent:  Chuck: b'_tvo', Checksum: 440, Counter: 15
Sent:  Chuck: b'c":0', Checksum: 239, Counter: 16
Sent:  Chuck: b'.0,"', Checksum: 172, Counter: 17
Sent:  Chuck: b'ccs8', Checksum: 369, Counter: 18
Sent:  Chuck: b'11_t',

In [ ]:
mc = canpack.MessageConstructor()
m_array = mc.create_message('message')
print(type(m_array[0]))
print(len(m_array))
arr = []
for m in m_array:
    #every 8 bits
    for i in range(0, len(m), 8):
        #every 8 bits
        arr.append((ctypes.c_ubyte * 8)(*[ctypes.c_ubyte(c) for c in m[i:i+8]]))
        
print(type(arr[0]))
    

In [ ]:
send_message("test")

In [91]:
message = 'Hello World'
test(message)

Sent:  Chuck: b'Hell', Checksum: 389, Counter: 1
0x48656c6c85010100
0x48656c6c85010100
[[CanalystMessage ID=0x300 TS=0x304f790 Data=48656c6c85010100]]
Sent:  Chuck: b'o Wo', Checksum: 341, Counter: 2
0x6f20576f55010200
No data received
[[]]
Sent:  Chuck: b'rld\x00', Checksum: 322, Counter: 3
0x726c640042010300


ValueError: non-hexadecimal number found in fromhex() arg at position 16

In [11]:
import canpack
import re
text = ''
with open('t.txt', 'r') as f:
    text = f.read()

# Regular expression pattern
pattern = r'Data=([a-fA-F0-9]+)'

# Find all matches
matches = re.findall(pattern, text)

# Print matches
for match in matches:
    print(match)
    
#print the number of matches
# print(len(matches))


#take each match and convert it to bytes
mc = canpack.MessageConstructor()
for match in matches:
    byte_array = bytes.fromhex(match)
    d = canpack.Message.fromBytesToMessage(byte_array)
    # print(d)
    #check if checksum is correct
    if d.checksum == mc.checksum(d.chuck.decode('ascii')):
        print(d.chuck.decode('ascii'),  end='')
print()

7b22646869010100
745f7465ac010200
6d70223a39010300
34392e30cb000400
2c2264681a010500
745f6875b0010600
6d223a32fb000700
392e302cc3000800
226363735b010900
3831315ff9000a00
636f322226010b00
3a302e30c8000c00
2c22636314010d00
733831310d010e00
5f74766fb8010f00
63223a30ef001000
2e302c22ac001100
6363733871011200
31315f7435011300
656d702264011400
3a32312ecb001500
33337d00e3001600
{"dht_temp":49.0,"dht_hum":29.0,"ccs811_co2":0.0,"ccs811_tvoc":0.0,"ccs811_temp":21.33} 


In [ ]:
#sending json
message = json.dumps({'name': 'John', 'age': 30})
test(message)


In [ ]:
#continously receive data
while True:
    try:
        data = dev.receive(1)
        print(data)
    except KeyboardInterrupt:
        dev.close()
        break